In [0]:
import dlt 
from pyspark.sql.functions import col

# Bronze Silver Gold Layers for Master Data

In [0]:
@dlt.table(
    comment="Bronze Layer - raw master data"
)

def bronze_customer():
    return spark.read.table("database_dev_2025.table_dev_2025")

In [0]:
@dlt.table(
    comment="Silver Layer - cleansed master data"
   
)

def silver_customer():
    df=dlt.read("bronze_customer")
    return df.filter(col("gender")=="Male")


In [0]:
@dlt.table(
    comment="Gold Layer - cleansed master data"
)

def gold_customer():
    df=dlt.read("silver_customer")
    return df.withColumn("is_Partner",col("Partner")=="Yes")

# Bronze Silver Gold Layers for Support Data

In [0]:
@dlt.table(
    comment= "Bronze layer for Support Customer data"
)

def support_customer():
    return spark.read.table("database_dev_2025.support_table_dev_2025")

In [0]:
@dlt.table(
    comment="Silver layer for Support Customer data"
)

def silver_support():
    df=dlt.read("support_customer")
    return df.filter(col("CVV_no")<=1500)

In [0]:
from pyspark.sql.functions import current_timestamp

@dlt.table(
    comment="Gold layer for Support Customer data"

)

def gold_support():
    df=dlt.read("silver_support")
    return df.withColumn("ingestion_time",current_timestamp())

In [0]:

@dlt.table(
    comment="Joining of these two tables"
)

def master_join_support():
    customer_df=dlt.read("gold_customer")
    support_df=dlt.read("gold_support")

    joined_df = customer_df.join(support_df, on="CustomerID",how="inner")

    return joined_df